What variables should be used for training. We look at the performance of the default training vs the training with each variable removed. If the performance improves when we remove a variable, we know that variable is making the training worse. Further, by looking at the change in performance we can rank the variables to first order.

# Initalization

In [1]:
from bdt_training_scikit_tools import load_trimmed_sample, default_training_variable_list, \
    test_train_samples, prep_samples, default_training, calc_performance, fraction, calcDFFilter
import matplotlib.pyplot as plt
plt.rc('font', size=14)
from matplotlib.colors import LogNorm
import pandas as pd
import numpy as np
import multiprocessing as mp
import itertools

# Load Data Samples

In [2]:
all_events_all = load_trimmed_sample("132")

Job 132:
  BIB: 800000 events
  Multijet: 800000 events
  Signal: 800000 events
  [800000, 800000, 504190]


In [3]:
def get_fraction_of_events(events, fractionToUse):
    '''Return a fraction of all events as training and testing samples.
    
    Args
        fractionToUse - If less than 1 then the fraction of each sample to use.
                        If > 1, then the total number of events
        
    Returns
        training - Training tripple of events (bib, mj, sig)
        testing - Testing tripple of events (bib, mj, sig)
    
    '''
    # Parse out the first parameter.
    fracsUC = (fractionToUse, fractionToUse, fractionToUse) if fractionToUse <= 1.0 else \
                (fractionToUse/len(e.index) for e in events)
    fracs = (f if f <= 1.0 else 1.0 for f in fracsUC)
    
    # Create the per event filter. We have to do this against each
    # of the three input samples.
    seqs = (fraction(f) for f in fracs)
    fracFilters = [calcDFFilter(df, seq) for df, seq in zip(events, seqs)]

    fraction_events = [dfi[1][dfi[0]] for dfi in zip(fracFilters,events)]
    return fraction_events

In [4]:
all_events = get_fraction_of_events(all_events_all, 200000)
print ([len(e.index) for e in all_events])

[200387, 199675, 198839]


# Determine Full Variable List
We have to determine what variable list we want to start with before we start removing them according to their performance. Reasons for removal:

- Weight variables are MC only things
- Variables that have nothing to do with phyisics (like run #)
- DR to closest track because it is used later in the analysis
- JetPhi because physics *shouldn't* depend on that

In [5]:
all_events_all[0].columns

Index(['RunNumber', 'Weight', 'WeightMCEvent', 'WeightXSection',
       'WeightFlatten', 'mc_Lxy', 'mc_Lz', 'MHTOverHT', 'JetPt', 'JetPhi',
       'CalRatio', 'JetEta', 'NTracks', 'SumPtOfAllTracks', 'MaxTrackPt',
       'EventNumber', 'JetET', 'JetWidth', 'JetDRTo2GeVTrack', 'EnergyDensity',
       'HadronicLayer1Fraction', 'JetLat', 'JetLong', 'FirstClusterRadius',
       'NumberOfClusters', 'ShowerCenter', 'BIBDeltaTimingM',
       'BIBDeltaTimingP', 'FirstCellTiming', 'InteractionsPerCrossing',
       'RPredictedLxy', 'RPredictedLz', 'PredictedLxy', 'PredictedLz',
       'PredictedLxyHighEta', 'PredictedLxyLowEta', 'PredictedLzHighEta',
       'PredictedLzLowEta'],
      dtype='object')

In [6]:
variable_list = set(all_events_all[0].columns) - set(['RunNumber', 'Weight', 'WeightMCEvent', 'WeightXSection', 'WeightFlatten', 'mc_Lxy', 'mc_Lz', 'MHTOverHT', 'JetPhi', 'EventNumber', 'InteractionsPerCrossing', 'JetDRTo2GeVTrack'])
variable_list

{'BIBDeltaTimingM',
 'BIBDeltaTimingP',
 'CalRatio',
 'EnergyDensity',
 'FirstCellTiming',
 'FirstClusterRadius',
 'HadronicLayer1Fraction',
 'JetET',
 'JetEta',
 'JetLat',
 'JetLong',
 'JetPt',
 'JetWidth',
 'MaxTrackPt',
 'NTracks',
 'NumberOfClusters',
 'PredictedLxy',
 'PredictedLxyHighEta',
 'PredictedLxyLowEta',
 'PredictedLz',
 'PredictedLzHighEta',
 'PredictedLzLowEta',
 'RPredictedLxy',
 'RPredictedLz',
 'ShowerCenter',
 'SumPtOfAllTracks'}

# Drop First Variable

In [7]:
%%writefile get_training_performance.py
from bdt_training_scikit_tools import load_default_samples, default_training_variable_list, \
    test_train_samples, prep_samples, default_training, calc_performance
    
def do_training (vlist):
    all_events, training_list = vlist
    return get_training_performance (all_events, training_list)
    
def get_training_performance (all_events, training_list):
    '''Run a training with the set of varaibles given. Return a performance table.'''
    
    # Split into testing and training samples
    train, test = test_train_samples(all_events)
        
    # Prep samples for training
    all_events, all_events_class, training_weight, evaluation_weight = prep_samples(train[0], train[1], train[2], training_variable_list=training_list)
    
    # Run training
    bdt = default_training(all_events, training_weight, all_events_class, estimators=200)
    
    # Create a thing of all the results
    return {tuple(training_list): calc_performance(bdt, test, training_variables = training_list)}

Overwriting get_training_performance.py


In [8]:
def all_but_one (vlist, number_to_drop = 1):
    '''Return vlist and vlist with each item removed
    
    Arguments:
        vlist - the source list or tuple
        number_to_drop - return a list that is len(vlist)-number_to_drop - all possible combinations
        
    Returns:
        A list of tuples that have len(vlist)-number_to_drop items.
    
    '''
    var_training_list = itertools.combinations(vlist, len(vlist)-number_to_drop)
    var_training_list = list(var_training_list) + [tuple(vlist)]
    return var_training_list

In [9]:
import get_training_performance
pool = mp.Pool(processes=5)

def unused_var(original_list, used_list):
    r = tuple(i for i in original_list if i not in used_list)
    r = r if len(r) != 0 else ('None',)
    return r

def calc_var_removal(all_events, training_list = default_training_variable_list):
    r_drop_first = pool.map(get_training_performance.do_training,
             [(all_events, tvar_list) for tvar_list in all_but_one(training_list)])

    one_dict = {}
    for kp in r_drop_first:
        one_dict.update(kp)

    return pd.DataFrame({unused_var(training_list, k):one_dict[k] for k in one_dict}).T


In [10]:
def remove_all_variables(all_events, training_list = variable_list):
    done = False
    count = 1
    while not done:
        print ("Iteration #{0}".format(count))
        print ('  Training with variable list:')
        print ('  ' + str(training_list))
        result = calc_var_removal(all_events, training_list=training_list)
        result_sorted = result.sort_values("HSSSsqrtB")
        print (result_sorted.HSSSsqrtB)
        last_var_name = result_sorted.index[-1][0]
        done = last_var_name == 'None'
        if not done:
            training_list = training_list - set([last_var_name])
    return training_list        

In [ ]:
%%time
#r_drop_first = calc_var_removal(all_events)

In [ ]:
#print (r_drop_first.sort_values("HSSSsqrtB").HSSSsqrtB)

In [ ]:
r = remove_all_variables(all_events, variable_list)

Iteration #1
  Training with variable list:
  {'NTracks', 'JetLong', 'SumPtOfAllTracks', 'NumberOfClusters', 'CalRatio', 'PredictedLxyHighEta', 'JetWidth', 'JetPt', 'PredictedLzHighEta', 'BIBDeltaTimingM', 'JetET', 'FirstClusterRadius', 'FirstCellTiming', 'EnergyDensity', 'JetLat', 'PredictedLxy', 'BIBDeltaTimingP', 'ShowerCenter', 'HadronicLayer1Fraction', 'RPredictedLxy', 'PredictedLxyLowEta', 'PredictedLzLowEta', 'PredictedLz', 'JetEta', 'MaxTrackPt', 'RPredictedLz'}


# Drop Second Variable

In [10]:
down_one_list = list(set(default_training_variable_list) - set(["CalRatio"]))
down_one_list

['NTracks',
 'JetPt',
 'ShowerCenter',
 'JetLong',
 'SumPtOfAllTracks',
 'PredictedLz',
 'EnergyDensity',
 'JetWidth',
 'BIBDeltaTimingM',
 'HadronicLayer1Fraction',
 'JetLat',
 'FirstClusterRadius',
 'MaxTrackPt',
 'PredictedLxy',
 'BIBDeltaTimingP']

In [11]:
r_drop_second = calc_var_removal(all_events, down_one_list)

In [12]:
r_drop_second.sort_values("HSSSsqrtB").HSSSsqrtB

JetPt                     33.560652
FirstClusterRadius        35.318773
EnergyDensity             35.756891
MaxTrackPt                36.731971
ShowerCenter              37.015712
BIBDeltaTimingP           37.628097
HadronicLayer1Fraction    37.944158
PredictedLz               38.863548
BIBDeltaTimingM           40.418985
NTracks                   40.636561
PredictedLxy              40.825376
JetLong                   41.256371
SumPtOfAllTracks          41.398124
None                      42.631388
JetLat                    42.849140
JetWidth                  43.121987
Name: HSSSsqrtB, dtype: float64

# Drop NTracks

In [13]:
down_two_list = list(set(down_one_list) - set(["JetWidth"]))
down_two_list

['EnergyDensity',
 'NTracks',
 'BIBDeltaTimingM',
 'JetPt',
 'HadronicLayer1Fraction',
 'ShowerCenter',
 'JetLat',
 'FirstClusterRadius',
 'BIBDeltaTimingP',
 'JetLong',
 'MaxTrackPt',
 'PredictedLxy',
 'SumPtOfAllTracks',
 'PredictedLz']

In [14]:
r_drop_third = calc_var_removal(all_events, down_two_list)

In [15]:
r_drop_third.sort_values("HSSSsqrtB").HSSSsqrtB

JetPt                     33.519731
BIBDeltaTimingM           35.923559
BIBDeltaTimingP           36.454895
JetLong                   38.038265
MaxTrackPt                38.789296
EnergyDensity             39.070243
ShowerCenter              39.447733
PredictedLz               39.746835
JetLat                    40.528748
HadronicLayer1Fraction    41.187335
FirstClusterRadius        41.811634
PredictedLxy              42.050360
SumPtOfAllTracks          42.644607
None                      43.121987
NTracks                   43.675357
Name: HSSSsqrtB, dtype: float64

In [19]:
down_three_list = list(set(down_two_list) - set(["NTracks"]))
down_three_list

['EnergyDensity',
 'BIBDeltaTimingM',
 'JetPt',
 'HadronicLayer1Fraction',
 'ShowerCenter',
 'JetLat',
 'FirstClusterRadius',
 'JetLong',
 'MaxTrackPt',
 'PredictedLxy',
 'BIBDeltaTimingP',
 'PredictedLz',
 'SumPtOfAllTracks']

In [20]:
r_drop_forth = calc_var_removal(all_events, down_three_list)

In [40]:
r_drop_forth.sort_values("HSSSsqrtB")[['HSSSsqrtB', 'HSSinHSS', 'MJinHSS', 'BIBinHSS']]

,HSSSsqrtB,HSSinHSS,MJinHSS,BIBinHSS
JetPt,33.441069,161973.0,2.344880e+07,11034.0
FirstClusterRadius,35.885192,162550.0,2.050928e+07,9115.0
ShowerCenter,36.262187,157932.0,1.895916e+07,9313.0
BIBDeltaTimingP,36.510289,161893.0,1.964281e+07,19091.0
HadronicLayer1Fraction,36.783127,161026.0,1.915541e+07,8965.0
PredictedLxy,36.793246,161843.0,1.933952e+07,9174.0
JetLat,37.816710,160183.0,1.793335e+07,8425.0
JetLong,37.823360,161054.0,1.812252e+07,8526.0
EnergyDensity,38.584012,162664.0,1.776416e+07,9139.0
BIBDeltaTimingM,38.722234,160869.0,1.723944e+07,19889.0
